In [1]:
import pysam
from umi_tools import umi_methods, network
from  functools import partial
import itertools
from tqdm import tqdm, tqdm_notebook

In [2]:
from umi_tools._dedup_umi import edit_distance

In [3]:
def breadth_first_search(node, adj_list):
    searched = set()
    found = set()
    queue = set()
    queue.update((node,))
    found.update((node,))

    while len(queue) > 0:
        node = queue.pop()
        found.update(adj_list[node])
        queue.update(adj_list[node])
        searched.update((node,))
        queue.difference_update(searched)

    return found

def recursive_search(node, adj_list):
    children = adj_list[node]
    children = [x for x in children if x not in recursive_search.component]
    for child in children:
        recursive_search.component.update((child,))
        recursive_search.component.update(
            recursive_search(child, adj_list))
    return recursive_search.component

def breadth_first_search_recursive(node, adj_list):
    try:
        recursive_search.component = set((node,))
        return recursive_search(node, adj_list)

    except RecursionError as error:
        U.info('Recursion Error: %s' % error)
        return breadth_first_search(node, adj_list)
    
def _get_adj_list_directional(umis, counts, threshold=1):
    ''' identify all umis within the hamming distance threshold
    and where the counts of the first umi is > (2 * second umi counts)-1'''

    adj_list = {umi: [] for umi in umis}
    for umi1, umi2 in itertools.combinations(umis, 2):
        if edit_distance(umi1, umi2) <= threshold:
            if counts[umi1] >= (counts[umi2]*2)-1:
                adj_list[umi1].append(umi2)
            if counts[umi2] >= (counts[umi1]*2)-1:
                adj_list[umi2].append(umi1)

    return adj_list

def _get_connected_components_adjacency_old(umis, graph, counts):
    ''' find the connected UMIs within an adjacency dictionary'''

    # TS: TO DO: Work out why recursive function does lead to same
    # final output. Then uncomment below

    #if len(graph) < 10000:
    #    self.search = breadth_first_search_recursive
    #else:
    #    self.search = breadth_first_search

    found = set()
    components = list()

    for node in sorted(graph, key=lambda x: counts[x], reverse=True):
        if node not in found:
            #component = self.search(node, graph)
            component = breadth_first_search(node, graph)
            found.update(component)
            components.append(component)

    return components

def _get_connected_components_adjacency_new(umis, graph, counts):
    ''' find the connected UMIs within an adjacency dictionary'''

    # TS: TO DO: Work out why recursive function does lead to same
    # final output. Then uncomment below

    #if len(graph) < 10000:
    #    self.search = breadth_first_search_recursive
    #else:
    #    self.search = breadth_first_search

    found = set()
    components = list()

    for node in sorted(graph, key=lambda x: counts[x], reverse=True):
        if node not in found:
            #component = self.search(node, graph)
            component = breadth_first_search_new(node, graph)
            found.update(component)
            components.append(component)

    return components

In [4]:
def breadth_first_search_new(node, adj_list):
    searched = set()
    queue = set()
    queue.update((node,))
    searched.update((node,))
    
    while len(queue) > 0:
        node = queue.pop()
        for next_node in adj_list[node]:
            if next_node not in searched:
                queue.update((next_node,))
                searched.update((next_node,))

    return searched

In [ ]:
< NB501001:62:HLWYHBGXX:1:11101:18506:2912_AGCTTT       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE<EAEAEEAAEEEEEEEEEAEEEEEEEEEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT
> NB501001:62:HLWYHBGXX:1:11101:18506:2912_AGCTTT       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE<EAEAEEAAEEEEEEEEEAEEEEEEEEEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT

> NB501001:62:HLWYHBGXX:1:11101:23689:4353_TTGGTG       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT
< NB501001:62:HLWYHBGXX:1:11101:23689:4353_TTGGTG       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT

> NB501001:62:HLWYHBGXX:1:11101:5151:5165_GGTGAG        0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEE/A/EE/EAEEEAEAEEEEEEEE<EEEEEEEEEEEEEAEEEEEEAAEEEEAEEEA/EE<<AAEEEAEEEEEEAEEEEEAEAEEAE6/EA<EEEAEEEE/A/A/<EEE/EEE<EE<EEEEEAEEEA<EEAE/EEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT
< NB501001:62:HLWYHBGXX:1:11101:5151:5165_GGTGAG        0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   AAAAAEEEEEEEEEEEEE/A/EE/EAEEEAEAEEEEEEEE<EEEEEEEEEEEEEAEEEEEEAAEEEEAEEEA/EE<<AAEEEAEEEEEEAEEEEEAEAEEAE6/EA<EEEAEEEE/A/A/<EEE/EEE<EE<EEEEEAEEEA<EEAE/EEA   NM:i:8  AS:i:125 XS:i:125 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT

> NB501001:62:HLWYHBGXX:1:11101:22013:2113_GGTGCA       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   /AAAA6///E//E/A///EEEEAAE//E//E/AEE/E//EA//E//E/EEEA/EE/A/EEEEEEE/EEEA/E/E<E/AA/66/EE//A/EA/E//EAA/6///A6/6A/AA<<//E/<<6/E6<6EEEE<AAA6E//AE///6/6E<E/E/   NM:i:9  AS:i:120 XS:i:120 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT
< NB501001:62:HLWYHBGXX:1:11101:22013:2113_GGTGCA       0       chr2    33141309        0       105M5D46M       *       0       0       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG   /AAAA6///E//E/A///EEEEAAE//E//E/AEE/E//EA//E//E/EEEA/EE/A/EEEEEEE/EEEA/E/E<E/AA/66/EE//A/EA/E//EAA/6///A6/6A/AA<<//E/<<6/E6<6EEEE<AAA6E//AE///6/6E<E/E/   NM:i:9  AS:i:120 XS:i:120 RG:Z:MOB3799_6_plasma_nr6       UG:i:9  BX:Z:TGCTTT



In [ ]:
['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'GGAGTT', 'TTGGTG', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'AGCTTT', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'CGGGAA', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGTGCA', 'GGGGTT', 'GGGATG', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']
['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGGATG', 'GGGGTT', 'GGTGCA', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']



In [ ]:
{'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}
{'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}


In [20]:
#old 
old_cluster = set(['GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'GGAGTT', 'TTGGTG', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'AGCTTT', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'CGGGAA', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'TGCTTT', 'GGTGCA', 'GGGGTT', 'GGGATG', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG'])
old_counts = {'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}

#new
new_cluster = set(['GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'TGCTTT', 'GGGATG', 'GGGGTT', 'GGTGCA', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG'])
new_counts = {'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}


In [23]:
old_final = ['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'GGAGTT', 'TTGGTG', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'AGCTTT', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'CGGGAA', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGTGCA', 'GGGGTT', 'GGGATG', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']
new_final = ['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGGATG', 'GGGGTT', 'GGTGCA', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']


In [24]:
old_final == new_final

False

In [ ]:
#old
set(['GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'GGAGTT', 'TTGGTG', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'AGCTTT', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'CGGGAA', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'TGCTTT', 'GGTGCA', 'GGGGTT', 'GGGATG', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG'])
set(['GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'TGCTTT', 'GGGATG', 'GGGGTT', 'GGTGCA', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG'])

{'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}
['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'GGAGTT', 'TTGGTG', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'AGCTTT', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'CGGGAA', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGTGCA', 'GGGGTT', 'GGGATG', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']



#new
{'GGGTTG': 1, 'TGTGTG': 1, 'GGATTT': 1, 'GGCTTT': 1, 'CCTCAT': 1, 'CGGTTT': 1, 'CGGGAA': 1, 'GGCGAG': 1, 'TTGGTG': 1, 'GAAGTT': 1, 'TGGGTT': 1, 'CGTGAA': 1, 'GGCGCA': 1, 'TGCGTT': 1, 'CGTGTG': 1, 'GGCGGA': 1, 'TTGGTC': 1, 'AGCTTT': 1, 'TGGGTG': 1, 'TAGGTG': 1, 'GCTGAT': 1, 'TATTTT': 1, 'GGAGTT': 1, 'GGTGCG': 1, 'CCTGAT': 1, 'GTTGCG': 1, 'CAGGTG': 1, 'GTTTCG': 1, 'TAAGTT': 1, 'GGGGTG': 1, 'GGGAAG': 1, 'GGTGAA': 1, 'TGGTTG': 1, 'GGTGTG': 1, 'GGTGCA': 1, 'GGGGTT': 1, 'GGGATG': 1, 'TGGTTT': 1, 'TGCTTC': 1, 'GGTGAT': 1, 'TACTTT': 1, 'TGCTTT': 2, 'GGTTCG': 1}
['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGGATG', 'GGGGTT', 'GGTGCA', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTTCG']


In [28]:
old_final_test = sorted(old_cluster, key=lambda x: old_counts[x],
                 reverse=True)

In [38]:
new_final_test = sorted(new_cluster, key=lambda x: new_counts[x],
                 reverse=True)

In [39]:
old_final_test == new_final_test

True

In [32]:
" ".join(new_final_test)

'TGCTTT GGGTTG TGTGTG GGATTT GGCTTT CCTCAT CGGTTT GGAGTT TTGGTG GAAGTT TGGGTT CGTGAA GGCGCA TGCGTT CGTGTG GGCGGA TTGGTC AGCTTT TGGGTG TAGGTG GCTGAT TATTTT CGGGAA GGTGCG CCTGAT GTTGCG CAGGTG GTTTCG TAAGTT GGGGTG GGTGTG GGTGAA TGGTTG GGGAAG GGGATG GGGGTT GGTGCA TGGTTT TGCTTC GGTGAT TACTTT GGTTCG'

In [21]:
new_counts == old_counts

True

In [22]:
new_cluster == old_cluster

True

In [5]:
umi_getter = partial(umi_methods.get_umi_read_id, sep="_")

In [19]:
'\|'.join(['TGCTTT', 'GGGTTG', 'TGTGTG', 'GGATTT', 'GGCTTT', 'CCTCAT', 'CGGTTT', 'CGGGAA', 'GGCGAG', 'AGCTTT', 'GAAGTT', 'GGGAAG', 'TGGGTT', 'CGTGAA', 'GGCGCA', 'TGCGTT', 'CGTGTG', 'GGCGGA', 'TTGGTC', 'TTGGTG', 'TGGGTG', 'TAGGTG', 'GCTGAT', 'TATTTT', 'GGAGTT', 'GGTGAA', 'CCTGAT', 'GTTGCG', 'CAGGTG', 'GTTTCG', 'TAAGTT', 'GGTGTG', 'GGTGCG', 'TGGTTG', 'GGGATG', 'GGGGTT', 'GGTGAG', 'CGTCAT', 'TGGTTT', 'TGCTTC', 'GGTGAT', 'TACTTT', 'GGGGTG', 'GGTGCA', 'GGTTCG'])

'TGCTTT\\|GGGTTG\\|TGTGTG\\|GGATTT\\|GGCTTT\\|CCTCAT\\|CGGTTT\\|CGGGAA\\|GGCGAG\\|AGCTTT\\|GAAGTT\\|GGGAAG\\|TGGGTT\\|CGTGAA\\|GGCGCA\\|TGCGTT\\|CGTGTG\\|GGCGGA\\|TTGGTC\\|TTGGTG\\|TGGGTG\\|TAGGTG\\|GCTGAT\\|TATTTT\\|GGAGTT\\|GGTGAA\\|CCTGAT\\|GTTGCG\\|CAGGTG\\|GTTTCG\\|TAAGTT\\|GGTGTG\\|GGTGCG\\|TGGTTG\\|GGGATG\\|GGGGTT\\|GGTGAG\\|CGTCAT\\|TGGTTT\\|TGCTTC\\|GGTGAT\\|TACTTT\\|GGGGTG\\|GGTGCA\\|GGTTCG'

In [6]:
infile = pysam.Samfile("https://s3-us-west-1.amazonaws.com/sauron-yeo/NSUN2.bam")
inreads = infile.fetch(reference="tRNA-Pro-TGG-2-1_withgenomeflank")

ValueError: could not open file (mode='r') - is it SAM/BAM format?

In [ ]:
infile = pysam.Samfile("/home/gpratt/projects/idr/analysis/umi_downsampling/452_CLIP_GAGATTCC-GGCTCTGA_L005_R1.A01_452_01_NSUN2.adapterTrim.round2.rmRep.sorted.adjusted.bam")
inreads = infile.fetch(reference="tRNA-Pro-TGG-2-1_withgenomeflank")

In [11]:
def _group_directional(clusters, adj_list, counts):
    ''' return groups for directional method'''

    observed = set()
    groups = []

    for cluster in clusters:
        if len(cluster) == 1:
            groups.append(list(cluster))
            observed.update(cluster)
        else:
            cluster = sorted(cluster, key=lambda x: counts[x],
                             reverse=True)
            # need to remove any node which has already been observed
            temp_cluster = []
            for node in cluster:
                if node not in observed:
                    temp_cluster.append(node)
                    observed.add(node)
            groups.append(temp_cluster)

    return groups

In [13]:
groups_old

[['GGGCGG']]

In [14]:
groups_new

[['GGGCGG']]

In [12]:
infile = pysam.Samfile("/home/gpratt/software/UMI-tools/tests/unmapped.bam")
inreads = infile.fetch(until_eof=True)

for bundle, read_events, status in umi_methods.get_bundles(inreads, 
                        ignore_umi=False,
                        paired=True,
                        soft_clip_threshold=4,
                        detection_method=None,
                        umi_getter=umi_getter):
    #print len(bundle.keys())
    umis = bundle.keys()
    counts = {umi: bundle[umi]["count"] for umi in umis}
    
    adj_list = _get_adj_list_directional(umis, counts, 1)
    old = _get_connected_components_adjacency_old(umis, adj_list, counts)
    new = _get_connected_components_adjacency_new(umis, adj_list, counts)
    
    groups_old = [list(x) for x in
              _group_directional(old, adj_list, counts)]
    groups_new = [list(x) for x in
              _group_directional(new, adj_list, counts)]
    
    if groups_old != groups_new:
        print "Error"

In [10]:
new

[{'GGGCGG'}]

In [9]:
old

[{'GGGCGG'}]

In [127]:
processor = network.ReadClusterer("directional")

In [90]:
# reads, umis, umi_counts, topologies, nodes = processor(bundle=bundle,
#                 threshold=1,
#                 stats=False,
#                 further_stats=False)

In [101]:
node = sorted(adj_list, key=lambda x: counts[x], reverse=True)[0]

In [119]:
len(breadth_first_search(node, adj_list))

7475

In [118]:
len(breadth_first_search_new(node, adj_list))

7475


  1%|▏         | 431/30010 [00:03<04:06, 119.78it/s]

100%|██████████| 30010/30010 [00:43<00:00, 690.43it/s]

In [124]:
recursive = _get_connected_components_adjacency(umis, adj_list, counts)

100%|██████████| 30010/30010 [04:23<00:00, 113.90it/s]


In [111]:
old = _get_connected_components_adjacency(umis, adj_list, counts)

100%|██████████| 30010/30010 [04:20<00:00, 115.42it/s]


In [125]:
old == recursive

True

In [102]:
len(breadth_first_search_recursive(node, adj_list))

7475

In [98]:
RecursionError()

NameError: name 'RecursionError' is not defined